# Нейросеть 

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split as tts

In [4]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    # Derivative of sigmoid
    return sigmoid(z) * (1 - sigmoid(z))


class Network:
    
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(i) for i in sizes[1:]]
        self.weights = [np.random.randn(j, i) for i, j in zip(sizes[:-1], sizes[1:])]

        
    def feedforward(self, x):
        for w, b in zip(self.weights, self.biases):
            x = sigmoid(np.dot(w, x) + b)
        return x
    
    
    def evaluate(self, test_data):
        """ test_data - tuples (x, y), x - input, y - desired output """
        res = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        return sum([int(x == y) for x, y in res]) / len(res)
    
    
    # Not needed for letter detection
    # needed for binary operations
    def evaluate2(self, test_data):
        cnt = 0
        for x, y in test_data:
            res = self.feedforward(x)
            if res == y:
                cnt += 1
            cnt+= int(np.argmax([1-res, res]) == y)
        return cnt / len(test_data)
    
    
    def backprop(self, x, y):
        
        delta_w = [np.zeros(w.shape) for w in self.weights]
        delta_b = [np.zeros(b.shape) for b in self.biases]
    
        # forward - get all the values
        act = x
        acts = [x]
        zs = []
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, act) + b
            zs.append(z)
            act = sigmoid(z)
            acts.append(act)
        
        acts = np.array(acts)
        zs = np.array(zs)
        
        # backward - reset all the values
        delta = (acts[-1] - y) * sigmoid_prime(zs[-1])
        delta_b[-1] = delta
        delta_w[-1] = np.dot(delta.reshape(-1, 1), acts[-2].reshape(1, -1))
        
        for l in range(2, self.num_layers):
            delta = np.dot(self.weights[-l+1].T, delta.reshape(-1, 1)) * sigmoid_prime(zs[-l]).reshape(-1, 1) 
            delta_b[-l] = delta.ravel()
            delta_w[-l] = np.dot(delta.reshape(-1, 1), acts[-l-1].reshape(1, -1))
        
        delta_w = np.array(delta_w)
        delta_b = np.array(delta_b)
        return delta_w, delta_b
    
    
    def train(self, train_data, epochs=10, eta=1):
        for epoch in range(epochs):
            for x, y in train_data:
                dw, db = self.backprop(x, y)
                self.biases = db
                self.weights -= eta * dw

## Проба - логическое умножение

In [637]:
def generateData(n=1000, sample_size=2, split=0.7):
    split_index = int(n * split)
    data = []
    for i in range(n):
        a = np.random.randint(2, size=sample_size)
        data.append((a, np.all(a).astype(np.int)))
    data = np.array(data)
    np.random.shuffle(data)
    return data[:split_index], data[split_index:]

In [638]:
def trainNet(net, train, lr=0.3):
    for t in train:
        dw, db = net.backprop(t[0], t[1])
        net.biases -= db
        net.weights -= lr * dw
        

N = [100, 200, 500, 700, 1000, 2000, 5000, 10000]
sizes = [(2, 1), (2, 2, 1), (2, 3, 1)]
for n in N:
    print(f'Sample size: {n}')
    for size in sizes:
        print(f'NetWork Size: {size}')
        net = Network(size)
        initAccs = []
        finAccs = []
        for i in range(10):
            train, test = generateData(n=n)
            initAcc = net.evaluate2(test)
            trainNet(net, train)
            finAcc = net.evaluate2(test)

            initAccs.append(initAcc)
            finAccs.append(finAcc)           
        print(f'Init Acc: {np.average(initAccs)}')
        print(f'Fin Acc: {np.average(finAccs)}\n')
        del(net)
        
    print('')

Sample size: 100
NetWork Size: (2, 1)
Init Acc: 0.8633333333333333
Fin Acc: 0.9133333333333333

NetWork Size: (2, 2, 1)
Init Acc: 0.6566666666666666
Fin Acc: 0.6900000000000001

NetWork Size: (2, 3, 1)
Init Acc: 0.8433333333333334
Fin Acc: 0.9099999999999999


Sample size: 200
NetWork Size: (2, 1)
Init Acc: 0.9233333333333335
Fin Acc: 0.9566666666666667

NetWork Size: (2, 2, 1)
Init Acc: 0.695
Fin Acc: 0.7699999999999999

NetWork Size: (2, 3, 1)
Init Acc: 0.8866666666666667
Fin Acc: 0.9183333333333333


Sample size: 500
NetWork Size: (2, 1)
Init Acc: 0.9253333333333333
Fin Acc: 1.0

NetWork Size: (2, 2, 1)
Init Acc: 0.8720000000000001
Fin Acc: 0.8966666666666667

NetWork Size: (2, 3, 1)
Init Acc: 0.9006666666666667
Fin Acc: 0.9746666666666666


Sample size: 700
NetWork Size: (2, 1)
Init Acc: 0.9450236966824643
Fin Acc: 1.0

NetWork Size: (2, 2, 1)
Init Acc: 0.9255924170616113
Fin Acc: 0.9497630331753555

NetWork Size: (2, 3, 1)
Init Acc: 0.9753554502369669
Fin Acc: 1.0


Sample size: 1

KeyboardInterrupt: 

## Распознавание букв А и С

### Подготовка данных

In [5]:
# converts an image to a np array of 0 and 1
def convert(file):
    image = Image.open(file)
    data = np.asarray(image)
    data2 = data[:, :, 3].ravel()
    return data, np.where(data2, 1, 0)


#  A 100x100, output = 0
A = []
files = os.listdir('a')
for file in files:
    if '.png' in file:
        _, x = convert('a/' + file)
        A.append(x)
A = np.array(A[:-2])

#  C 100x100, output = 1
C = []
files = os.listdir('c')
for file in files:
    if '.png' in file:
        _, x = convert('c/' + file)
        C.append(x)
C = np.array(C[:-1])

In [6]:
# Desired ouputs for A and C correspondingly
Data = []
for a in A: Data.append((a, [1., 0.]))
for c in C: Data.append((c, [0., 1.]))
Data = np.array(Data)

In [21]:
# Train / Test (70/30)
np.random.shuffle(Data)
train, test = Data[:140], Data[140:]

### Создание нейросети 100х70х2

In [8]:
net = Network((100, 70, 2))
net.train(train_data=train, epochs=800, eta=0.003)
print('Accuracy: ', net.evaluate(test))
del(net)

Accuracy:  0.9833333333333333


### Тестирование на разном разбиении

In [715]:
epochs = [350, 500, 750, 1000, 1500, 5000]
W = []
B = []
for eps in epochs:
    print(f'Number of epochs: {eps}')
    accs = []
    for i in range(10):
        np.random.shuffle(Data)
        train, test = Data[:140], Data[140:]
        net = Network((100, 70, 2))
        net.train(train_data=train, epochs=eps, eta=0.003)
        acc = net.evaluate(test)
        accs.append(acc)
        print(f'{i+1}. Accuracy: {acc}')
        
        if acc > 0.96:
            W.append((net.weights, acc, eps))
            B.append((net.biases, acc, eps))
            
        del(net)
    print('Mean accuracy: ', np.average(accs))
    print('____________\n\n')

Number of epochs: 350
1. Accuracy: 1.0
2. Accuracy: 0.95
3. Accuracy: 0.9333333333333333
4. Accuracy: 1.0
5. Accuracy: 0.43333333333333335
6. Accuracy: 0.9666666666666667
7. Accuracy: 0.9333333333333333
8. Accuracy: 0.9166666666666666
9. Accuracy: 0.9333333333333333
10. Accuracy: 0.9666666666666667
Mean accuracy:  0.9033333333333333
____________


Number of epochs: 500
1. Accuracy: 0.95
2. Accuracy: 0.9666666666666667
3. Accuracy: 0.9666666666666667
4. Accuracy: 0.9666666666666667
5. Accuracy: 0.95
6. Accuracy: 0.9666666666666667
7. Accuracy: 0.95
8. Accuracy: 0.9666666666666667
9. Accuracy: 0.95
10. Accuracy: 0.95
Mean accuracy:  0.9583333333333333
____________


Number of epochs: 750
1. Accuracy: 0.9
2. Accuracy: 0.9166666666666666
3. Accuracy: 0.95
4. Accuracy: 0.9833333333333333
5. Accuracy: 0.95
6. Accuracy: 0.95
7. Accuracy: 1.0
8. Accuracy: 0.9833333333333333
9. Accuracy: 0.9833333333333333
10. Accuracy: 0.9333333333333333
Mean accuracy:  0.9549999999999998
____________


Number

### Параметры с точностью более 0.98

In [716]:
W = np.array(W)
B = np.array(B)
res = W[:, [1, 2]]
weights = W[:, 0][res[:, 0] > 0.98]
biases = B[:, 0][res[:, 0] > 0.98]

best_acc = 0
j = 1
for w, b in zip(weights, biases):
    net = Network((100, 70, 2))
    net.weights = w
    net.biases = b
    accs = []
    for i in range(100):
        np.random.shuffle(Data)
        train, test = Data[:140], Data[140:]
        acc = net.evaluate(test)
        accs.append(acc)
    del(net)  
    mean_acc = round(np.average(accs), 2)
    min_acc = round(np.min(accs), 2)
    max_acc = np.max(accs)
    
    if mean_acc > best_acc:
        best_acc = mean_acc
        best_w = w
    
    
    print(f'{j}. Mean Acc: {mean_acc}, Min Acc: {min_acc}')
    j += 1
    
# print(best_acc, best_w)

1. Mean Acc: 0.99, Min Acc: 0.95
2. Mean Acc: 1.0, Min Acc: 1.0
3. Mean Acc: 0.99, Min Acc: 0.97
4. Mean Acc: 1.0, Min Acc: 1.0
5. Mean Acc: 0.99, Min Acc: 0.97
6. Mean Acc: 0.99, Min Acc: 0.97
7. Mean Acc: 0.99, Min Acc: 0.97
8. Mean Acc: 1.0, Min Acc: 1.0
9. Mean Acc: 1.0, Min Acc: 0.98
10. Mean Acc: 0.99, Min Acc: 0.97
11. Mean Acc: 1.0, Min Acc: 1.0
12. Mean Acc: 1.0, Min Acc: 0.98
13. Mean Acc: 1.0, Min Acc: 0.98
14. Mean Acc: 0.99, Min Acc: 0.97
15. Mean Acc: 1.0, Min Acc: 1.0
16. Mean Acc: 1.0, Min Acc: 1.0
17. Mean Acc: 0.99, Min Acc: 0.98
18. Mean Acc: 1.0, Min Acc: 1.0
19. Mean Acc: 1.0, Min Acc: 0.98
20. Mean Acc: 1.0, Min Acc: 0.98
21. Mean Acc: 1.0, Min Acc: 1.0
22. Mean Acc: 1.0, Min Acc: 0.98


In [717]:
best_params = [(weights[i], biases[i]) for i in [4, 8, 9, 12]]

### Тестирование на новых данных

https://www.pixilart.com/draw?ref=home-page

In [718]:
A = []
files = os.listdir('a_test')
for file in files:
    if '.png' in file:
        _, x = convert('a_test/' + file)
        A.append(x)
A = np.array(A)

C = []
files = os.listdir('c_test')
for file in files:
    if '.png' in file:
        _, x = convert('c_test/' + file)
        C.append(x)
C = np.array(C)

Data2 = []
for a in A: Data2.append((a, [1., 0.]))
for c in C: Data2.append((c, [0., 1.]))
Data2 = np.array(Data2)

In [719]:
# pars = zip(weights, biases)
pars = zip(W[:, 0], B[:, 0])
net = Network((100, 70, 2))

accs = []
for i, p in enumerate(pars):
    net.weights, net.biases = p
    acc = net.evaluate(Data2)
    accs.append(acc)
    if acc > 0.9:
        print(i+1, acc)
    
print(f'Mean Accuracy: {np.mean(accs)}')

2 0.918918918918919
Mean Accuracy: 0.8222768222768222


## Создание нейросети 100х250х70х2

### Обучение

In [720]:
sizes = (100, 250, 70, 2)
W = []
B = []
epochs = [500, 750, 1000]
for eps in epochs:
    print(f'Number of epochs: {eps}')
    accs = []
    for i in range(10):
        np.random.shuffle(Data)
        train, test = Data[:140], Data[140:]
        
        net = Network(sizes)
        net.train(train_data=train, epochs=eps, eta=0.003)
        acc = net.evaluate(test)
        accs.append(acc)
        
        if acc > 0.95:
            W.append(net.weights)
            B.append(net.biases)
        
        print(f'{i+1}. Accuracy: {acc}')
        del(net)
        
    mean_acc = np.mean(accs)
    min_acc = np.min(accs)
        
    print(f'Mean Acc: {mean_acc}, Min Acc: {min_acc} \n____________\n')

Number of epochs: 500
1. Accuracy: 0.9666666666666667
2. Accuracy: 0.9833333333333333
3. Accuracy: 0.4166666666666667
4. Accuracy: 0.9
5. Accuracy: 0.9833333333333333
6. Accuracy: 0.9333333333333333
7. Accuracy: 0.9166666666666666
8. Accuracy: 0.9833333333333333
9. Accuracy: 0.95
10. Accuracy: 0.9166666666666666
Mean Acc: 0.8949999999999999, Min Acc: 0.4166666666666667 
____________

Number of epochs: 750
1. Accuracy: 0.9166666666666666
2. Accuracy: 0.95
3. Accuracy: 0.9666666666666667
4. Accuracy: 0.4666666666666667
5. Accuracy: 0.9333333333333333
6. Accuracy: 0.9666666666666667
7. Accuracy: 0.9833333333333333
8. Accuracy: 0.9833333333333333
9. Accuracy: 0.9333333333333333
10. Accuracy: 0.9666666666666667
Mean Acc: 0.9066666666666666, Min Acc: 0.4666666666666667 
____________

Number of epochs: 1000
1. Accuracy: 0.9166666666666666
2. Accuracy: 0.9666666666666667
3. Accuracy: 0.9666666666666667
4. Accuracy: 0.9333333333333333
5. Accuracy: 0.9333333333333333
6. Accuracy: 0.95
7. Accurac

### Тестирование на новых данных

In [725]:
net = Network(sizes)

accs = []
ba = 0
for w, b in zip(W, B):
    net.weights = w
    net.biases = b
    acc = net.evaluate(Data2)
    accs.append(acc)
    print(acc)
    ba = max(ba, acc)

print(f'\nMean accuracy: {np.mean(accs)}')
print(f'Best accuracy: {ba}')
del(net)

0.8108108108108109
0.6756756756756757
0.8108108108108109
0.7837837837837838
0.8378378378378378
0.7567567567567568
0.8378378378378378
0.7567567567567568
0.7837837837837838
0.8108108108108109
0.8378378378378378
0.9459459459459459
0.7837837837837838

Mean accuracy: 0.8024948024948025
Best accuracy: 0.9459459459459459


### За тренировочные данные взяты 200 рисунков. Тестовые - 37 новых.

In [733]:
newW = []
newB = []

epochs = [200, 250, 450]
for eps in epochs:
    print(f'Number of epochs: {eps}')
    
    accs = []
    ba = 0
    for i in range(10):
        net = Network(sizes)
        net.train(train_data=Data, epochs=eps, eta=0.01)
        acc = net.evaluate(Data2)
        accs.append(acc)
        ba = max(ba, acc)
        
        if acc > 0.9:
            newW.append(net.weights)
            newB.append(net.biases)
        
        print(f'{i+1}. Acc: {acc}')
        del(net)
        
    print(f'\n Mean acc: {np.mean(accs)}')
    print(f'Best acc: {np.mean(ba)}, \n_________\n')

Number of epochs: 200
1. Acc: 0.8378378378378378
2. Acc: 0.6756756756756757
3. Acc: 0.7567567567567568
4. Acc: 0.7837837837837838
5. Acc: 0.8378378378378378
6. Acc: 0.7027027027027027
7. Acc: 0.9459459459459459
8. Acc: 0.8108108108108109
9. Acc: 0.8108108108108109
10. Acc: 0.8108108108108109

 Mean acc: 0.7972972972972973
Best acc: 0.9459459459459459, 
_________

Number of epochs: 250
1. Acc: 0.8648648648648649
2. Acc: 0.6756756756756757
3. Acc: 0.8378378378378378
4. Acc: 0.918918918918919
5. Acc: 0.8378378378378378
6. Acc: 0.7837837837837838
7. Acc: 0.8378378378378378
8. Acc: 0.8108108108108109
9. Acc: 0.8648648648648649
10. Acc: 0.7837837837837838

 Mean acc: 0.8216216216216216
Best acc: 0.918918918918919, 
_________

Number of epochs: 450
1. Acc: 0.8918918918918919
2. Acc: 0.8108108108108109
3. Acc: 0.8378378378378378
4. Acc: 0.8648648648648649
5. Acc: 0.8378378378378378
6. Acc: 0.8648648648648649
7. Acc: 0.8378378378378378
8. Acc: 0.7837837837837838
9. Acc: 0.7567567567567568
10. A

In [729]:
BestW = W
BestB = B